In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

%matplotlib inline

In [4]:
import gensim
import nltk as nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora

%matplotlib inline

In [5]:
# download the stop words
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
# also set a random state
rs = 123

In [7]:
#dimensionality reduction using stop words
stop_words = set(stopwords.words('english'))

In [8]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [9]:
course_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/datasets/course_processed.csv"
course_content_df = pd.read_csv(course_url)

In [10]:
course_content_df.iloc[0, :]

COURSE_ID                                               ML0201EN
TITLE          robots are coming  build iot apps with watson ...
DESCRIPTION    have fun with iot and learn along the way  if ...
Name: 0, dtype: object

In [11]:
# Merge TITLE and DESCRIPTION title
course_content_df['course_texts'] = course_content_df[['TITLE', 'DESCRIPTION']].agg(' '.join, axis=1)
course_content_df = course_content_df.reset_index()
course_content_df['index'] = course_content_df.index
course_content_df

,index,COURSE_ID,TITLE,DESCRIPTION,course_texts
0,0,ML0201EN,robots are coming build iot apps with watson ...,have fun with iot and learn along the way if ...,robots are coming build iot apps with watson ...
1,1,ML0122EN,accelerating deep learning with gpu,training complex deep learning models with lar...,accelerating deep learning with gpu training c...
2,2,GPXX0ZG0EN,consuming restful services using the reactive ...,learn how to use a reactive jax rs client to a...,consuming restful services using the reactive ...
3,3,RP0105EN,analyzing big data in r using apache spark,apache spark is a popular cluster computing fr...,analyzing big data in r using apache spark apa...
4,4,GPXX0Z2PEN,containerizing packaging and running a sprin...,learn how to containerize package and run a ...,containerizing packaging and running a sprin...
...,...,...,...,...,...
302,302,excourse89,javascript jquery and json,in this course we ll look at the javascript l...,javascript jquery and json in this course w...
303,303,excourse90,programming foundations with javascript html ...,learn foundational programming concepts e g ...,programming foundations with javascript html ...
304,304,excourse91,front end web development with react,this course explores javascript based front en...,front end web development with react this cour...
305,305,excourse92,introduction to web development,this course is designed to start you on a path...,introduction to web development this course is...


In [12]:
course_content_df.iloc[0, :]

index                                                           0
COURSE_ID                                                ML0201EN
TITLE           robots are coming  build iot apps with watson ...
DESCRIPTION     have fun with iot and learn along the way  if ...
course_texts    robots are coming  build iot apps with watson ...
Name: 0, dtype: object

In [13]:
a_course = course_content_df.iloc[0, :]['course_texts']
a_course

'robots are coming  build iot apps with watson  swift  and node red have fun with iot and learn along the way  if you re a swift developer and want to learn more about iot and watson ai services in the cloud  raspberry pi   and node red  you ve found the right place  you ll build iot apps to read temperature data  take pictures with a raspcam  use ai to recognize the objects in those pictures  and program an irobot create 2 robot  '

In [14]:
course_content_df.iloc[0]['course_texts']
course_content_df.index[0]

0

In [15]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [16]:
#tokenize all courses
bows_df = pd.DataFrame()
# filter using stop words i.e. aka dimensionality reduction
stop_words = set(stopwords.words('english'))
for i in course_content_df.index:
    # instead of course_texts column (which is a union of title & description)
    # using only title as course_texts seem to be taking lot of time for execution!!!
    tokenized_course = word_tokenize(course_content_df.loc[i]['course_texts'])
    # Remove English stop words and numbers
    tokenized_course = [w for w in tokenized_course if (not w.lower() in stop_words) and (not w.isnumeric())]
    #print(tokenized_course)
    # Create a token dictionary for the course
    tokens_dict = gensim.corpora.Dictionary([tokenized_course])
    # Generate BoW features for each course
    course_bow = tokens_dict.doc2bow(tokenized_course)
    #print(course_bow)
    # print each token and its count
    # For each token index, print its bow value (word count)
    for token_index, token_bow in course_bow:
        token = tokens_dict.get(token_index)
        bow_dicts = {"doc_index": [token_index],
                      "doc_id": [course_content_df.loc[i]['COURSE_ID']],
                    "token": [token],  
                    "bow": [token_bow]}
        temp = pd.DataFrame(bow_dicts)
        bows_df = d = pd.concat([bows_df, temp])
        #print(f"--Token: '{token}', Count:{token_bow}")

In [17]:
bows_df

,doc_index,doc_id,token,bow
0,0,ML0201EN,ai,2
0,1,ML0201EN,along,1
0,2,ML0201EN,apps,2
0,3,ML0201EN,build,2
0,4,ML0201EN,cloud,1
...,...,...,...,...
0,39,excourse93,specialization,1
0,40,excourse93,third,1
0,41,excourse93,throughout,1
0,42,excourse93,together,1


In [18]:
#Course Simolarity Calculations - consine, jaccard index, or euclidean distance
from scipy.spatial.distance import euclidean, jaccard, cosine
from nltk import ngrams

%matplotlib inline

In [19]:
#generate BoW features (token counts) for these two courses 
#(or using tokens_dict.doc2bow() method provided in nltk, similar to what we did in the previous lab).
def generate_sparse_bow(course):
    bow_vector = []
    words = course.split()
    for token in tokens:
        if token in words:
            bow_vector.append(1)
        else:
            bow_vector.append(0)
    return bow_vector

In [20]:
course_content_df[course_content_df['COURSE_ID'] == 'ML0101ENv3']

,index,COURSE_ID,TITLE,DESCRIPTION,course_texts
158,158,ML0101ENv3,machine learning with python,machine learning can be an incredibly benefici...,machine learning with python machine learning ...


In [21]:
# associated BoW features
ml_course = bows_new_df[bows_df['doc_id'] == 'ML0101ENv3']
ml_course

NameError: name 'bows_new_df' is not defined

In [ ]:
#BoW feature vector is in vertical format but normally feature vectors are in horizontal format. 
#One way to transpose the feature vector from vertical to horizontal is to use the Pandas pivot() method:
ml_courseT = ml_course.pivot(index=['doc_id'], columns='token').reset_index(level=[0])
ml_courseT

In [ ]:
#To compare the BoWs of any two courses, which normally have a different set of tokens, we need to create a union token set and then transpose them. We have provided a method called pivot_two_bows as follows:
def pivot_two_bows(basedoc, comparedoc):
    base = basedoc.copy()
    base['type'] = 'base'
    compare = comparedoc.copy()
    compare['type'] = 'compare'
    # Append the two token sets vertically
    join = pd.concat([base, compare])
    # Pivot the two joined courses
    joinT = join.pivot(index=['doc_id', 'type'], columns='token').fillna(0).reset_index(level=[0, 1])
    # Assign columns
    joinT.columns = ['doc_id', 'type'] + [t[1] for t in joinT.columns][2:]
    return joinT

In [ ]:
course1 = bows_df[bows_df['doc_id'] == 'ML0151EN']
course2 = bows_df[bows_df['doc_id'] == 'ML0101ENv3']

In [ ]:
bow_vectors = pivot_two_bows(course1, course2)
bow_vectors

In [ ]:
bow_vectors.iloc[0, 2:]
bow_vectors.iloc[1, 2:]


In [ ]:
similarity = 1 - cosine(bow_vectors.iloc[0, 2:], bow_vectors.iloc[1, 2:])
similarity

In [ ]:
course_content_df[course_content_df['COURSE_ID'] == 'ML0101ENv3']

In [ ]:
course_content_df

In [ ]:
# pick a sampl course for similarity calculations & comparisons
base = bows_df[bows_df['doc_id'] == 'ML0101ENv3']
similarity_threshold = 0.5
sims = pd.DataFrame()
#for i in bows_df.index:
for i in course_content_df.index:
    sim = []
    doc_id = course_content_df.loc[i]['COURSE_ID']    
    compare = bows_df[bows_df['doc_id'] == doc_id]
    bow_vectors = pivot_two_bows(base, compare)

    bow1 = bow_vectors.iloc[0, 2:]
    bow2 = bow_vectors.iloc[1, 2:]
    
    cos_similarity = cosine(bow1, bow2)
    ecludean_similarity = euclidean(bow1, bow2)
    jackard_similarity = jaccard(bow1, bow2)

    sim_min = np.min([cos_similarity, ecludean_similarity, jackard_similarity])
    sim_max = np.max([cos_similarity, ecludean_similarity, jackard_similarity])
    if (sim_min > similarity_threshold or sim_max > similarity_threshold):
        #print('course: {}:cosine similarity {}'.format(bows_df.loc[i]['doc_id']))
        sim = {  "doc_id": [doc_id],
                 "cos-sim": [cos_similarity],  
                 "jack-sim": [jackard_similarity],
                 "eclu-sim" : [ecludean_similarity]
            }
        temp = pd.DataFrame(sim)
        sims = pd.concat([sims, temp])


In [ ]:
sims.describe

In [ ]:
sims.groupby('doc_id').size()

In [ ]:
sims[sims['cos-sim'] < 1]

In [ ]:
# Create course id to index and index to id mappings
def get_doc_dicts(bow_df):
    grouped_df = bow_df.groupby(['doc_index', 'doc_id']).max().reset_index(drop=False)
    idx_id_dict = grouped_df[['doc_id']].to_dict()['doc_id']
    id_idx_dict = {v: k for k, v in idx_id_dict.items()}
    del grouped_df
    return idx_id_dict, id_idx_dict

In [ ]:
idx_id_dict, id_idx_dict = get_doc_dicts(bows_df)
idx1 = id_idx_dict["ML0151EN"]
idx2 = id_idx_dict["ML0101ENv3"]
print(f"Course 1's index is {idx1} and Course 2's index is {idx2}")